# Data Wrangling avec Pandas

Une grande partie du métier de Data Scientist, consiste à nétoyer, arranger des données afin de les exploiter. 

Maîtriser cette partie du processus est essentiel : les données que vous aurait l'occasion d'analyser seront rarement sous la forme d'un DataFrame propre.
    
A la fin de cette partie, vous saurez :
    
- Fusionner deux DataFrame avec la méthode **merge**
- Retravailler, faire pivoter deux datasets avec les méthodes **stack** et **pivot**
- Remplacer des valeurs à partir d'un dictionnaire grâce à la méthode **map**

## Fusionner

Fusionner consiste à rassembler les lignes de deux DataFrames en utilisant une clé commune. 

Prenons l'exemple suivant : 

In [1]:
import pandas as pd

Prenons deux sets de données. 

Le premier contient des id de commandes. Le second la correspondance entre id client et nom et prenoms. 

In [2]:
orders_dict = {'customer_id':['3','2','1','4'],
          'order_value':[40,35,50,45],
          'order_id':['4001','4002','4003','4004']}

customers_dict = {'customer_id':['1','2','3','4','5'],
                  'nom':['CAMPAN','DOE','MUSK','JOBS','FRANKO'],
                  'prenom':['Bernard','John','Elon','Steve','James']}

Nous pouvons créer deux DataFrames pour chacun de ces dictionnaires, soit : 

In [ ]:
orders = pd.DataFrame(orders_dict,
                       columns=['customer_id','order_value'],
                       index = orders_dict['order_id'])

customers = pd.DataFrame(customers_dict,
                         index = customers_dict['customer_id'])

Nous pouvons **fusionner** ces deux DataFrames en utilisant la méthode **merge**. 

Nous spécifions la clé commune ***customer_id*** au sein du paramètre **on**

In [ ]:
pd.merge(orders,customers, on='customer_id')

Plusieurs choses sont à noter : 

- La méthode merge a supprimé l'index que nous avions créé
- Une seule des deux colonnes a été utilisée
- Le client qui n'était pas présent dans le DataFrame orders n'est pas présent dans le résultat final 

Par défaut, Pandas réalise une fusion dite **inner**.

Il existe plusieurs de fusion entre deux sets de données. Les principales peuvent se résumer au sein du graphe ci-dessous : 

<img src='files/images/merge.png'>

Nous pouvons spécifier à Pandas le type de fusion que nous souhaitons grâce au paramètre **how**. 

Si nous souhaitons par exemple réaliser une fusion à droite, nous écrirons : 

In [ ]:
pd.merge(orders,customers,on='customer_id',how='right')

Le client James FRANKO, n'a ici pas de d'order_value associée.

Nous pouvons également spécifier une fusion sur un Index grâce aux méthodes **left_index** et **right_index**. 

Dans notre cas, le DataFrame orders dispose d'un index, nous pouvons donc écrire : 

In [ ]:
pd.merge(orders,customers, left_on = 'customer_id', right_index=True)

A noter que dans ce cas : 
    
- Nous conservons l'index qui a été utilisé
- Nous conservons les deux colonnes (customer_id et customer_id_x)

## Reshaping

Pandas permet de retravailler des données tabulaires pour mieux les exploiter. 

In [3]:
import numpy as np

Le reshaping intervient souvent après une opération de grouping sur plusieurs clés. 

Prenons le cas du dataset ***coffee_shop*** : 

In [4]:
coffee = pd.read_csv("data/coffee_shop.csv")

coffee.head(1)

,Area Code,Date,Market,Market Size,Product,Product Line,Product Type,State,Type,Inventory,Budget COGS,Budget Margin,Budget Profit,Budget Sales,COGS,Margin,Marketing,Profit,Sales,Total Expenses
0,985,12/1/13 0:00,South,Small Market,Colombian,Beans,Coffee,Louisiana,Regular,845,50,90,70,140,49,71,13,68,128,25


Si nous souhaitons connaître le nombre de ventes par **Market** et **Market Size**, nous écrirons : 

In [5]:
coffee_market = coffee['Sales'].groupby([coffee['Market'],coffee['Market Size']]).sum()

coffee_market

Market   Market Size 
Central  Major Market    152579
         Small Market    112466
East     Major Market    138260
         Small Market     40316
South    Major Market     37410
         Small Market     66516
West     Major Market     96892
         Small Market    175372
Name: Sales, dtype: int64

**coffee_market** est une Série dispose d'un **MultiIndex**. **Market** et **Market Size** sont deux index hiérarchisés, cf :

In [6]:
coffee_market.index

MultiIndex(levels=[[u'Central', u'East', u'South', u'West'], [u'Major Market', u'Small Market']],
           labels=[[0, 0, 1, 1, 2, 2, 3, 3], [0, 1, 0, 1, 0, 1, 0, 1]],
           names=[u'Market', u'Market Size'])

Plusieurs choses sont à noter pour cet Index : 

- Nous avons deux **levels** ou **niveaux**, hiérarchisés
- Ces **levels** disposent de noms (Market et Market Size), permettant de les appeler
- Pandas a également ajouté des valeurs d'indice, appelé **labels**

Les méthodes **stack** et **unstack** permettent de faire pivoter ces indices de lignes à colonnes, ou de colonnes à lignes :  

In [ ]:
coffee_unstacked = coffee_market.unstack()
coffee_unstacked

La méthode stack réalise l'inverse de la méthode **unstack** : 

In [ ]:
coffee_unstacked.stack()

Par défaut, les méthodes stack et unstack utilisent le **level** le plus bas pour réaliser un pivot (ici il s'agit de Market Size). 
                                                                                                  
Nous pouvons toutefois spécifier le **level**, en utilisant le nom ou la position du **level** dans l'Index. 

Le level **Market** étant en première position, la rotation de lignes à colonnes sur le level **Market** peut s'écrire :

In [7]:
coffee_market.unstack(0)

Market,Central,East,South,West
Market Size,,,,
Major Market,152579,138260,37410,96892
Small Market,112466,40316,66516,175372


In [ ]:
ou encore : 

In [8]:
coffee_market.unstack('Market')

Market,Central,East,South,West
Market Size,,,,
Major Market,152579,138260,37410,96892
Small Market,112466,40316,66516,175372


Il arrive que les données soient stockées au format **stacked** au sein de bases de données. 

On souhaite dans ce cas réaliser la même rotation mais à partir d'une colonne (et non d'Index). 

Prenons le set **datalong.csv** : 

In [9]:
data = pd.read_csv("data/datalong.csv")

data.head(3)

,date,item,value
0,1959-03-31 00:00:00,realgdp,2710.349
1,1959-03-31 00:00:00,infl,0.000
2,1959-03-31 00:00:00,unemp,5.800


Dans ce cas, nous disposons de trois 'items' : realgdp, infl et unemp. 

La ligne date est répétée autant de fois qu'il y a d'items distincts. 

Si nous souhaitons faire une rotation, en créant trois nouvelles colonnes, correspondant à chacune des valeurs de items, nous pouvons utiliser la méthode **pivot** :

In [ ]:
data_pivoted = data.pivot('date','item')

data_pivoted.head(3)

La méthode **pivot** prend au moins deux arguments. Le premier est l'Index à utiliser pour les lignes, Le second est l'index à utiliser pour les colonnes. 

Le troisième paramètre est optionnel. Il permet de déterminer quelle valeur à utiliser pour remplir le DataFrame

## Transformer des données

Nous allons voir dans cette partie comment supprimer des doublons ou transformer des données à partir d'un dictionnaire

Les doublons sont omniprésents dans une analyse de données, la méthode **duplicate** permet de rapidement repérer et supprimer des doublons. Considérons le DataFrame suivant : 

In [ ]:
doublons_dict = {'col1':['a','b','a','b','b','a'],
            'col2':[1,1,1,2,2,3]}

doublons = pd.DataFrame(doublons_dict)

La méthode **duplicated** permet de renvoyer un booléenn spécifiant si chacune des lignes est un doublon ou non :

In [ ]:
doublons.duplicated()

Par défaut, la méthode **duplicated** se base sur l'ensemble des colonnes du dataset. 

Nous pouvons spécifier la colonne à considérer en paramètre, soit : 

In [ ]:
doublons.duplicated('col1')

La méthode **drop_duplicates** permet par ailleurs de supprimer directement toutes les lignes en doublon : 

In [ ]:
doublons.drop_duplicates()

La transformation de données passe également souvent par un mapping de valeurs avec un dictionnaire. 

Supposons que nous ayons un dictionnaire villes_customers qui associe à chaque customer_id, la ville de résidence :

In [ ]:
villes_customers = {'1':'Paris','2':'Seattle','3':'San Francisco','4':'Cupertino','5':'Los Angeles'}

Nous pouvons réaliser un **mapping** de la colonne customer_id du DataFrame customers grâce à la fonction **map** :

In [ ]:
customers['customer_id'].map(villes_customers)

## A vous : 

Nous allons travailler dans cet exercice sur le set de données **Movielens** - https://movielens.org/. 
    
Movielens est issu d'un groupe de recherche en Machine Learning de l'Université du Minnesota. Le service permet à un utilisateur de disposer de recommandations de films.

Le set de données contient 1M+ d'avis, que 6k+ utilisateurs de Movielens ont donné sur 4k films depuis 2000. Trois fichiers sont disponibles : 

- **movies.dat** : MovieID::Title::Genres
- **users.dat** : UserID::Gender::Age::Occupation::Zip-code

    Les correspondances pour les âges sont : 
    
    *  1:  "Under 18"
	* 18:  "18-24"
	* 25:  "25-34"
	* 35:  "35-44"
	* 45:  "45-49"
	* 50:  "50-55"
	* 56:  "56+"
    
    Les correspondances pour les occupations sont : 
    
    *  0:  "other" or not specified
	*  1:  "academic/educator"
	*  2:  "artist"
	*  3:  "clerical/admin"
	*  4:  "college/grad student"
	*  5:  "customer service"
	*  6:  "doctor/health care"
	*  7:  "executive/managerial"
	*  8:  "farmer"
	*  9:  "homemaker"
	* 10:  "K-12 student"
	* 11:  "lawyer"
	* 12:  "programmer"
	* 13:  "retired"
	* 14:  "sales/marketing"
	* 15:  "scientist"
	* 16:  "self-employed"
	* 17:  "technician/engineer"
	* 18:  "tradesman/craftsman"
	* 19:  "unemployed"
	* 20:  "writer"
    
- **ratings.dat** : UserID::MovieID::Rating::Timestamp

Nous créons trois DataFrames : users, ratings et movies : 

In [ ]:
# pass in column names for each CSV
u_cols = ['user_id', 'sex', 'age', 'occupation', 'zip_code']
users = pd.read_table('data/movielens/users.dat', sep='::', names=u_cols)

r_cols = ['user_id', 'movie_id', 'rating', 'unix_timestamp']
ratings = pd.read_table('data/movielens/ratings.dat', sep='::', names=r_cols)

m_cols = ['movie_id', 'title', 'genre']
movies = pd.read_table('data/movielens/movies.dat', sep='::', names=m_cols, usecols=range(5))

- Créez un DataFrame unique en fusionnant les users, ratings et movies
- Quels sont les 5 films qui ont été le plus noté ? 
- Quels sont les 5 films qui ont la meilleure note moyenne ayant été noté plus de 100 fois ? 
- Quelle est la tranche d'âge notant le moins bien ? 
- Les hommes notent ils moins bien que les femmes ? 
- [Avancé] Quels sont les films ayant la plus grande différence de notes entre les hommes et les femmes ? 